In [120]:
import requests
import json
from pathlib import Path
import os
import numpy as np
import pickle
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import clear_output, display
import pandas as pd
from random import randrange
from hazm import *

In [121]:
sparqlEndpoint = "http://farsbase.net:8890/sparql"
sparql = SPARQLWrapper(sparqlEndpoint)

In [122]:
def fixURI(uri):
    splited = uri.split('/')
    if('relation' in splited[len(splited)-1]):
        fixedURI = ''
        for i in range (0, len(splited)-2):
            fixedURI += splited[i] + '/'
        fixedURI += splited[len(splited)-2]
        return fixedURI
    else:
        return uri

In [123]:
def getNameFromURI(uri):
    splited = fixURI(uri).split('/')
    splitedEntityName = splited[len(splited)-1].split('_')
    entityName = ''
    for i in range (0, len(splitedEntityName)-1):
        entityName += splitedEntityName[i] + ' '
    entityName += splitedEntityName[len(splitedEntityName)-1]
    return entityName
    

In [124]:
dfs = pd.read_excel("sentenceTemplates.xlsx", sheet_name='Sheet1')

In [125]:
class sentenceTemplate:
    sType = ''
    sHead = []
    usedSHead = []
    sE1 = ''
    sE1URI = ''
    sE2 = ''
    sE2URI = ''
    sDC = []
    usedSDC = []
    sMid = []
    usedSMid = []
    sTail = []
    usedSTail = []
    sRelation = ''
    
    def __init__(self, st, shead, se1, se1URI, se2, se2URI, sdc, smid, stail, sr):
        self.sType = st
        self.sHead = shead
        self.usedSHead = []
        if(len(self.sHead) == 0):
            self.sHead.append("*")
        self.sE1 = se1
        self.sE1URI = se1URI
        self.sE2 = se2
        self.sE2URI = se2URI
        self.sDC = sdc
        self.usedSDC = []
        if(len(self.sDC) == 0):
            self.sDC.append("*")
        self.sMid = smid
        self.usedSMid = []
        if(len(self.sMid) == 0):
            self.sMid.append("*")
        self.sTail = stail
        self.usedSTail = []
        if(len(self.sTail) == 0):
            self.sTail.append("*")
        self.sRelation = sr
        
    def __init__(self, st, shead, se1, se2, sdc, smid, stail, sr):
        self.sType = st
        self.sHead = shead
        self.usedSHead = []
        if(len(self.sHead) == 0):
            self.sHead.append("*")
        self.sE1 = se1
        self.sE2 = se2
        self.sDC = sdc
        self.usedSDC = []
        if(len(self.sDC) == 0):
            self.sDC.append("*")
        self.sMid = smid
        self.usedSMid = []
        if(len(self.sMid) == 0):
            self.sMid.append("*")
        self.sTail = stail
        self.usedSTail = []
        if(len(self.sTail) == 0):
            self.sTail.append("*")
        self.sRelation = sr
        

In [126]:
templates = []
for i in range (0, len(dfs)):
    st = sentenceTemplate(dfs['Type'][i], str(dfs['Head'][i]).split(','), dfs['Fisrt Entity'][i], dfs['Second Entity'][i], str(dfs['Dependent Clause'][i]).split(','), str(dfs['Mid'][i]).split(','), str(dfs['Tail'][i]).split(','), dfs['Relation'][i])
    templates.append(st)

In [127]:
def makeSentence(st):
    print(len(st.sHead))
    print(len(st.usedSHead) >= len(st.sHead))
    if (len(st.usedSHead) >= len(st.sHead)):
        return None
    headIndex = len(st.usedSHead)
    if (len(st.usedSDC) >= len(st.sDC)):
        st.usedSDC = []
        st.usedSHead.append(st.sHead[len(st.usedSHead)])
    dcIndex = len(st.usedSDC)
    if (len(st.usedSMid) >= len(st.sMid)):
        st.usedSMid = []
        st.usedSDC.append(st.sDC[len(st.usedSDC)])
    midIndex = len(st.usedSMid)
    if (len(st.usedSTail) >= len(st.sTail)):
        st.usedSTail = []
        st.usedSMid.append(st.sMid[len(st.usedSMid)])
    tailIndex = len(st.usedSTail)
    st.usedSTail.append(st.sTail[len(st.usedSTail)])
    sentence = str(st.sHead[headIndex]) + " " + str(st.sE1) + " " + str(st.sDC[dcIndex]) + " " + str(st.sMid[midIndex]) + " " + str(st.sE2) + " " + str(st.sTail[tailIndex])
    sentence = sentence.replace('nan', '').replace('  ', ' ').replace('  ', ' ').replace(' ، ', '، ').replace('*', '')
    if(sentence[0] == ' '):
        sentence = sentence[1:]
    if(sentence[len(sentence) - 1] == ' '):
        sentence = sentence[0: len(sentence) - 1]
    return sentence

In [128]:
# -*- coding: utf-8 -*-
def isEnglishChar(s):
    if(s in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"):
        return True
    else:
        return False
    
def isEnglish(s):
    hasEn = False
    for i in range (0, len(s)):
        if(isEnglishChar(s[i])):
            hasEn = True
    return hasEn

def isURIValid(uri):
    if("http" in uri):
        return True
    else:
        return False
    
dfs = pd.read_excel("sentenceTemplates.xlsx", sheet_name='Sheet1')
templates = []
for i in range (0, len(dfs)):
    st = sentenceTemplate(dfs['Type'][i], str(dfs['Head'][i]).split(','), dfs['Fisrt Entity'][i], dfs['Second Entity'][i], str(dfs['Dependent Clause'][i]).split(','), str(dfs['Mid'][i]).split(','), str(dfs['Tail'][i]).split(','), dfs['Relation'][i])
    templates.append(st)
    
dfss = pd.read_excel("Properties_refined_two_side_entity_filterd.xlsx", sheet_name=None)
relations = []
for i in range (0, len(dfss)):
    relations.append(dfss['URL'][i])
print((str(dfss['translatedLabel'][relations.index(relations[0])])))

generatedRelations = []

for i in range (0, len(templates)):
    stringQuery = """select ?s ?o WHERE {
        ?s <""" + str(templates[i].sRelation) + """>  ?o
        } limit 1000"""
    sparql.setQuery(stringQuery)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for j in range(9*len(results['results']['bindings'])//10, 10*len(results['results']['bindings'])//10):
        subjectURI = results['results']['bindings'][j]['s']['value']
        objectURI = results['results']['bindings'][j]['o']['value']
        
        if(isURIValid(subjectURI) and isURIValid(objectURI)):
            pass
        else:
            continue
        if(isEnglish(getNameFromURI(subjectURI))):
            continue
        templates[i].sE1 = getNameFromURI(subjectURI)
        templates[i].sE1RUI = fixURI(subjectURI)
        if(isEnglish(getNameFromURI(objectURI))):
            continue
        templates[i].sE2 = getNameFromURI(objectURI)
        templates[i].sE2RUI = fixURI(objectURI)
        
        newRelation = {}
        newRelation['sentence'] = makeSentence(templates[i])
        if(not(newRelation['sentence'] == None)):
            head = {}
            head['word'] = templates[i].sE1
            head['start'] = newRelation['sentence'].find(head['word'])
            head['length'] = len(head['word'])
            head['id'] = templates[i].sE1RUI
            newRelation['head'] = head
            tail = {}
            tail['word'] = templates[i].sE2
            tail['start'] = newRelation['sentence'].find(tail['word'])
            tail['length'] = len(tail['word'])
            tail['id'] = templates[i].sE2RUI
            newRelation['tail'] = tail
            relation = {}
    #         print(templates[i].sR1)
            relation['word'] = str(dfss['translatedLabel'][relations.index(templates[i].sRelation)])
            relation['id'] = templates[i].sRelation
            newRelation['relation'] = relation
            newRelation['why'] = 'generated'

            if(not(newRelation in generatedRelations)):
                generatedRelations.append(newRelation)
            
#         print(makeSentence(templates[i]))

شهرستان
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1


2
False
2
False
2
False
2
False
2
False
2
False
2
False
2
False
2
False
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
1
False
1
False
1
False
1
False
1
False
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True

1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
False
1
False
1
False
1
False
1
False
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
Tr

1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
False
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
1
True
2
False
2
False
2
False
2
False
2
False
2
False
2
False
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2
True
2

In [129]:
print(len(generatedRelations))

300


In [130]:
resultDataFile = open("generated-simple-new-patterns-10.json", "w", encoding='utf-8')
resultDataFile.write(str(json.dumps(generatedRelations, indent=4, sort_keys=True, ensure_ascii=False)))
resultDataFile.close()